In [1]:
import snowflake.connector as sf
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM


In [14]:
from flask import Flask
app = Flask(__name__)

In [2]:
# Define the S3 bucket and Snowflake parameters
import snowflake.connector as sf
#Define the S3 bucket and Snowflake parameters
snowflake_account = 'qg08548.central-india.azure'
snowflake_user = 'FYP3'
snowflake_password = '!Q2w3e4r5t'
snowflake_database = 'COAPDATA'
snowflake_schema = 'PUBLIC'
snowflake_warehouse = 'COMPUTE_WH'
snowflake_table = 'daily_consumption'


In [8]:
snowflake_account = 'BF48652.ap-south-1.aws'
snowflake_user = 'FYP2'
snowflake_password = '!Q2w3e4r5t'
snowflake_database = 'COAPDATA'
snowflake_schema = 'PUBLIC'
snowflake_warehouse = 'COMPUTE_WH'
snowflake_table = 'mlview'

In [4]:
# Define the model parameters
n_input = 10
n_features = 1
model_file_name = 'model.h5'


In [5]:
# load the initial LSTM model
model = load_model('model.h5')

In [6]:
def train():    
    # Connect to Snowflake and retrieve the latest data
    conn = sf.connect(user=snowflake_user, password=snowflake_password, account=snowflake_account)
    cursor = conn.cursor()
    cursor.execute(f"SELECT * FROM {snowflake_database}.{snowflake_schema}.{snowflake_table} ORDER BY Date DESC")
    data = cursor.fetchall()
    cursor.close()
    conn.close()
    df = pd.DataFrame(data, columns=['Date', 'consumption'])
    df.set_index('Date', inplace=True)
    df.sort_index(inplace=True)
    
    # Scale the data using the same scaler used for training
    scaler = MinMaxScaler()
    scaler.fit(df)
    scaled_data = scaler.transform(df)

    # Update the model with the new data
    generator = TimeseriesGenerator(scaled_data, scaled_data, length=n_input, batch_size=1)
    
    # take the last 10 and reshape
    last_scaled_batch = scaled_data[-10:]
    last_scaled_batch = last_scaled_batch.reshape((1, n_input, n_features))
    
    
    model.fit(generator, epochs=1, verbose=0)
    
    prediction = model.predict(last_scaled_batch)
    true_prediction = scaler.inverse_transform(prediction)[0][0]
    
    snowflake_conn = sf.connect(
            user='FYP3',
            password='!Q2w3e4r5t',
            account='qg08548.central-india.azure',
            warehouse='COMPUTE_WH',
            database='COAPDATA',
            schema='PUBLIC'
    )


    cursor = snowflake_conn.cursor()
    cursor.execute(f"INSERT INTO predict_table (date, prediction) VALUES (DATEADD(DAY, 1, (SELECT MAX(date) FROM daily_consumption)), {true_prediction})")
    cursor.close()
    snowflake_conn.close()
    
    return 'Training completed successfully'


In [9]:
train()

'Training completed successfully'